In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import torch
print(torch.__version__)
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
import random
import cv2
import numpy as np
!pip install pyefd
import pyefd
from google.colab.patches import cv2_imshow
!pip install cairocffi
import cairocffi as cairo
import struct
from struct import unpack

1.12.1+cu113
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 3.6 MB/s 
  Created wheel for cairocffi: filename=cairocffi-1.4.0-py3-none-any.whl size=88775 sha256=0664989eca961d8c996bb928f257570418851306c53d28f30ab7f8cc804210a0
  Stored in directory: /root/.cache/pip/wheels/7a/2b/da/aec872f95d2c24105496ef149a9a576f52daf686f8f2127541
Successfully built cairocffi


Section 1: Fourier Descriptors through linear classifier

In [3]:
# Env vars
torch.use_deterministic_algorithms(False)

# Const vars
LOAD_PATH = '/content/drive/My Drive/Fourier/Saved Models/Subset25-2 MLP vector single-wide.pt'
RAND_SEED = 0
DEVICE = "cuda"

IMG_SIDE = 256
NUM_CLASSES = 25
EPOCHS = 10
LEARNING_RATE = 0.001
BATCH_SIZE = 512
LOSS_FN = nn.CrossEntropyLoss()
FOURIER_ORDER = 10

In [4]:
def get_step(deltaX, deltaY):
  angle = np.arctan2(deltaY, deltaX)
  pi_over_8 = np.pi / 8
  if -pi_over_8 <= angle < pi_over_8:
    return (1, 0)
  elif pi_over_8 <= angle < 3 * pi_over_8:
    return (1, 1)
  elif 3 * pi_over_8 <= angle < 5 * pi_over_8:
    return (0, 1)
  elif 5 * pi_over_8 <= angle < 7 * pi_over_8:
    return (-1, 1)
  elif angle >= 7 * pi_over_8 or angle < -7 * pi_over_8:
    return (-1, 0)
  elif -7 * pi_over_8 <= angle < -5 * pi_over_8:
    return (-1, -1)
  elif -5 * pi_over_8 <= angle < -3 * pi_over_8:
    return (0, -1)
  else:
    return (1, -1)

def complete_contour(vector_img):
  complete_contour = []
  partial_contour = [point for stroke in vector_img for point in zip(*stroke)]
  if len(partial_contour) > 0:
    partial_contour.append(partial_contour[0])
    for i in range(len(partial_contour) - 1):
      x0, y0 = partial_contour[i]
      x1, y1 = partial_contour[i+1]
      if (x0, y0) == (x1, y1):
        continue
      complete_contour.append([x0, y0])
      deltaX = x1 - x0
      deltaY = y1 - y0
      while(max(abs(deltaX), abs(deltaY)) > 1):
        x_offset, y_offset = get_step(deltaX, deltaY)
        x0 += x_offset
        y0 += y_offset
        complete_contour.append([x0, y0])
        deltaX = x1 - x0
        deltaY = y1 - y0
  return np.asarray(complete_contour)

means = np.asarray([[1.00000000e+00,  1.70292625e-19,  1.37157261e-19, -2.97352773e-02],
 [ 3.93857845e-02,  1.14573042e-02,  1.11273905e-02, -1.27501456e-02],
 [ 1.26057258e-02,  1.27047894e-02,  2.20446418e-03, -5.17296120e-03],
 [ 4.60867346e-03,  2.54431263e-04, -2.24875991e-03, -2.43568670e-03],
 [-1.29011658e-03,  7.29537187e-04,  3.29306713e-04, -6.94848183e-05],
 [-2.20461684e-03, -4.26032423e-04,  3.70747798e-04,  6.67042619e-04],
 [ 1.06105248e-03,  5.59965312e-04, -2.48033261e-05, -3.27838867e-04],
 [ 6.92991267e-04,  7.77918380e-05,  1.26802111e-04, -4.82832044e-04],
 [ 2.05231401e-04, -1.29655496e-04, -1.44908870e-04, -2.86502180e-04],
 [-1.48524853e-04, -2.01465364e-04,  2.28534063e-04,  3.38029409e-04]])

stdevs = np.asarray([[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 3.94827207e-01],
 [6.15715499e-01, 6.31045641e-01, 5.56262445e-01, 5.77054655e-01],
 [4.44723755e-01, 4.34617210e-01, 4.14754373e-01, 4.09847350e-01],
 [3.13965726e-01, 3.06039399e-01, 2.93664393e-01, 2.88944168e-01],
 [2.20598291e-01, 2.20428499e-01, 2.08627310e-01, 2.08939754e-01],
 [1.67779200e-01, 1.70621159e-01, 1.61133731e-01, 1.60840793e-01],
 [1.35504574e-01, 1.33753600e-01, 1.28539543e-01, 1.29087092e-01],
 [1.08136940e-01, 1.08601151e-01, 1.04438585e-01, 1.03887367e-01],
 [9.21341521e-02, 9.22050837e-02, 8.90123677e-02, 8.74814102e-02],
 [7.74686779e-02, 7.69117614e-02, 7.27067303e-02, 7.27266299e-02]])
 
# transform functions - take sketch image, return torch tensor of descriptors
def fourier_transform(vector_img, is_test):
  contour = complete_contour(vector_img)

  if is_test: 
    angle = random.random() * 60 - 30
    deltaX = random.randint(-3, 3)
    deltaY = random.randint(-3, 3)

    # linear algebra to apply randomly generated transformations to the digit contour
    theta = np.radians(angle)
    sin, cos = np.sin(theta), np.cos(theta)
    rotation = np.array(((cos, -sin), (sin, cos)))
    translation = np.array((deltaX, deltaY))
    contour = np.transpose(np.dot(rotation, np.transpose(contour))) + translation
  
  coeffs = pyefd.elliptic_fourier_descriptors(contour, order=FOURIER_ORDER, normalize=True)
  coeffs = (coeffs - means) / stdevs
  return torch.from_numpy(coeffs.flatten()).float()

# helper method to find class based on imgset index
def find_class(idx, num_list):
  class_id = 0
  sum = num_list[class_id]
  while idx >= sum:
    class_id += 1
    sum += num_list[class_id]
  return class_id

# deterministic worker re-seeding
def seed_worker(worker_id):
  worker_seed = torch.initial_seed() % 2**32
  np.random.seed(worker_seed)
  random.seed(worker_seed)

# custom dataset for quickdraw
class QuickdrawDataset(Dataset):
  def __init__(self, imgs, nums, is_test):
    self.imgs = imgs
    self.nums = nums
    self.len = sum(nums)
    self.is_test = is_test

  def __len__(self):
    return self.len

  def __getitem__(self, idx):
    img = self.imgs[idx]
    x = fourier_transform(img, self.is_test)
    y = find_class(idx, self.nums)
    return x, y


class MLP(torch.nn.Module):
  def __init__(self):
    super(MLP, self).__init__()
    self.fc1 = nn.Linear(FOURIER_ORDER * 4, 1024)
    self.fc2 = nn.Linear(1024, 1024)
    self.fc3 = nn.Linear(1024, 1024)
    self.head = nn.Linear(1024, NUM_CLASSES)
    self.relu = nn.ReLU()


  def forward(self, x):
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    x = self.relu(x)
    x = self.fc3(x)
    x = self.relu(x)
    return self.head(x)


def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train() # put the model in train mode
    total_loss = 0
    total_correct = 0
    # for each batch in the training set compute loss and update model parameters
    for batch, (x, y) in enumerate(dataloader):
      x, y = x.to(DEVICE), y.to(DEVICE)
      # Compute prediction and loss
      out = model(x)
      loss = loss_fn(out, y)

      # Backpropagation to update model parameters
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # print current training metrics for user
      y, out, loss = y.to("cpu"), out.to("cpu"), loss.to("cpu")
      loss_val = loss.item()
      if batch % 20 == 0:
          current = (batch + 1) * BATCH_SIZE
          print(f"loss: {loss_val:>7f}  [{current:>5d}/{size:>5d}]")

      pred = out.argmax(dim=1, keepdim=True)
      correct = pred.eq(y.view_as(pred)).sum().item()
      total_correct += correct
      total_loss += loss_val
      # print(f"train loss: {loss_val:>7f}   train accuracy: {correct / BATCH_SIZE:>7f}   [batch: {batch + 1:>3d}/{(size // BATCH_SIZE) + 1:>3d}]")      
    print(f"\nepoch avg train loss: {total_loss / ((size // BATCH_SIZE) + 1):>7f}   epoch avg train accuracy: {total_correct / size:>7f}")
      
def eval_loop(dataloader, model):
  model.eval()
  size = len(dataloader.dataset)
  with torch.no_grad():
    total_correct = 0
    for x, y in dataloader:
      x, y = x.to(DEVICE), y.to(DEVICE)
      out = model(x)
      y, out = y.to("cpu"), out.to("cpu")
      pred = out.argmax(dim=1, keepdim=True)
      total_correct += pred.eq(y.view_as(pred)).sum().item()

    accuracy = total_correct / size
    return accuracy


In [ ]:
# define methods for unpacking Quickdraw .bin files
def unpack_drawing(file_handle):
  file_handle.read(15)
  n_strokes, = unpack('H', file_handle.read(2))
  image = []
  for i in range(n_strokes):
      n_points, = unpack('H', file_handle.read(2))
      fmt = str(n_points) + 'B'
      x = unpack(fmt, file_handle.read(n_points))
      y = unpack(fmt, file_handle.read(n_points))
      image.append((x, y))

  return image


def unpack_drawings(filename):
  imageset = []
  with open(filename, 'rb') as f:
      while True:
          try:
              imageset.append(unpack_drawing(f))
          except struct.error:
              break
  return imageset

train_dir = '/content/drive/My Drive/Fourier/Quickdraw Dataset Small/Train/'
test_dir = '/content/drive/My Drive/Fourier/Quickdraw Dataset Small/Test/'
train_imgs = []
test_imgs = []
train_nums = []
test_nums = []
# list_of_classes = ["arm", "brain", "circle", "ear", "elbow", "eye", 
#                    "face", "finger", "foot", "hand", "hexagon", "knee", 
#                    "leg", "line", "mouth", "nose", "octagon", 
#                    'skull', 'square', 'squiggle', 'star', 'toe', 'tooth', 
#                    'triangle', 'zigzag']
list_of_classes = ["aircraft carrier", "airplane", "alarm clock", "ambulance", 
                   "angel", "ant", "anvil", "apple", "arm", "asparagus", "axe", 
                   "backpack", "banana", "bandage", "barn", "baseball bat", 
                   "baseball", "basket", "basketball", "bathtub", "beach", "bear", 
                   "beard", "bed", "bee"]

In [ ]:
# load dataset
for item in list_of_classes:
  train_folder = train_dir + item + '.bin'
  test_folder = test_dir + item + '.bin'
  train_drawings = unpack_drawings(train_folder)
  train_imgs += train_drawings
  train_nums.append(len(train_drawings))
  test_drawings = unpack_drawings(test_folder)
  test_imgs += test_drawings
  test_nums.append(len(test_drawings))

In [ ]:
DEVICE = "cpu"
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)
g = torch.Generator()
g.manual_seed(RAND_SEED)

def get_confusion_mat(dataloader, model):
  model.eval()
  size = len(dataloader.dataset)
  conf_mat = np.zeros((NUM_CLASSES, NUM_CLASSES))
  with torch.no_grad():
    total_correct = 0
    for x, y in dataloader:
      x, y = x.to(DEVICE), y.to(DEVICE)
      out = model(x)
      y, out = y.to("cpu"), out.to("cpu")
      pred = out.argmax(dim=1, keepdim=True)
      y = y.view_as(pred)
      total_correct += pred.eq(y).sum().item()
      y, pred = y.numpy(), pred.numpy()
      indices = np.concatenate((y, pred), axis=1)
      for i in range(indices.shape[0]):
        conf_mat[indices[i, 0], indices[i, 1]] += 1

    accuracy = total_correct / size
    print(f"test accuracy: {accuracy:>7f}")
    return conf_mat

eval_data = QuickdrawDataset(test_imgs, test_nums, is_test=False)
eval_loader = DataLoader(eval_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

model = MLP()
checkpoint = torch.load(LOAD_PATH, map_location=torch.device(DEVICE))
model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)

random.seed(RAND_SEED)
conf_mat = get_confusion_mat(dataloader=eval_loader,model=model)
conf_mat_norm = conf_mat * (255 / 100)
cv2_imshow(conf_mat_norm)

diag = np.diagonal(conf_mat)
sums = np.asarray(test_nums)
accuracies = diag / sums
acc_sorted_idxs = np.argsort(accuracies)
for i in range(NUM_CLASSES):
  print(list_of_classes[acc_sorted_idxs[i]] + " accuracy " + str(accuracies[acc_sorted_idxs[i]]))

print("\n")

diag_mat = np.multiply(np.identity(NUM_CLASSES), conf_mat)
conf_mat_nodiag = conf_mat - diag_mat
conf_mat_sorted = np.argsort(conf_mat_nodiag, axis=None)
for i in range(1, 6):
  print("class " + list_of_classes[conf_mat_sorted[-i] // NUM_CLASSES] + " misclassified as " + list_of_classes[conf_mat_sorted[-i] % NUM_CLASSES])

test accuracy: 0.563826


hexagon accuracy 0.19
knee accuracy 0.28
foot accuracy 0.34
brain accuracy 0.37
leg accuracy 0.39
squiggle accuracy 0.42
toe accuracy 0.42424242424242425
elbow accuracy 0.45
octagon accuracy 0.5
eye accuracy 0.54
nose accuracy 0.55
finger accuracy 0.56
skull accuracy 0.56
mouth accuracy 0.58
tooth accuracy 0.59
face accuracy 0.6
zigzag accuracy 0.65
ear accuracy 0.66
arm accuracy 0.67
circle accuracy 0.69
star accuracy 0.76
triangle accuracy 0.78
square accuracy 0.82
hand accuracy 0.82
line accuracy 0.9


class hexagon misclassified as octagon
class circle misclassified as octagon
class brain misclassified as skull
class squiggle misclassified as zigzag
class eye misclassified as mouth


In [ ]:
# seed RNGs
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)

# create datasets
train_data = QuickdrawDataset(train_imgs, train_nums, is_test=False)
# eval_fourier_data = QuickdrawDataset(test_imgs, test_nums, is_test=False)
test_data = QuickdrawDataset(test_imgs, test_nums, is_test=True)

# create dataloaders
g = torch.Generator()
g.manual_seed(RAND_SEED)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
# eval_fourier_loader = DataLoader(eval_fourier_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

# init model and optimizer
model = MLP()
# checkpoint = torch.load(LOAD_PATH, map_location=torch.device(DEVICE))
# model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# optim.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
epoch = 0
print("\n\n\nFourier order is: "+str(FOURIER_ORDER)+"\n\n\n")

# train for EPOCHS number of epochs then evaluate on test data with affine transformations
eval_loop(dataloader=test_loader,model=model)
SAVE_PATH = '/content/drive/My Drive/Fourier/Saved Models/Subset25-2 MLP vector single-wide.pt'
for i in range(epoch, EPOCHS):
    print("Epoch " + str(i + 1) + "\n")
    train_loop(dataloader=train_loader,model=model,loss_fn=LOSS_FN,optimizer=optim)
    # eval_loop(dataloader=eval_fourier_loader,model=model)
    torch.save({
                'epoch': i + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optim.state_dict()
                }, SAVE_PATH)
    print("\n-------------------------------\n")
random.seed(RAND_SEED)
accuracies = []
for i in range(30):
  accuracies.append(eval_loop(dataloader=test_loader,model=model))
accuracies = np.asarray(accuracies)
mean = np.mean(accuracies)
std = np.std(accuracies)
print(mean)
print(std)

In [ ]:
# seed RNGs
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)

# create datasets
train_data = QuickdrawDataset(train_imgs, train_nums, is_test=False)
# eval_fourier_data = QuickdrawDataset(test_imgs, test_nums, is_test=False)
test_data = QuickdrawDataset(test_imgs, test_nums, is_test=True)

# create dataloaders
g = torch.Generator()
g.manual_seed(RAND_SEED)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
# eval_fourier_loader = DataLoader(eval_fourier_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

# init model and optimizer
model = MLP()
# checkpoint = torch.load(LOAD_PATH, map_location=torch.device(DEVICE))
# model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# optim.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
epoch = 0
print("\n\n\nFourier order is: "+str(FOURIER_ORDER)+"\n\n\n")

# train for EPOCHS number of epochs then evaluate on test data with affine transformations
eval_loop(dataloader=test_loader,model=model)
SAVE_PATH = '/content/drive/My Drive/Fourier/Saved Models/Subset25 MLP vector single-wide.pt'
for i in range(epoch, EPOCHS):
    print("Epoch " + str(i + 1) + "\n")
    train_loop(dataloader=train_loader,model=model,loss_fn=LOSS_FN,optimizer=optim)
    # eval_loop(dataloader=eval_fourier_loader,model=model)
    torch.save({
                'epoch': i + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optim.state_dict()
                }, SAVE_PATH)
    print("\n-------------------------------\n")
random.seed(RAND_SEED)
accuracies = []
for i in range(30):
  accuracies.append(eval_loop(dataloader=test_loader,model=model))
accuracies = np.asarray(accuracies)
mean = np.mean(accuracies)
std = np.std(accuracies)
print(mean)
print(std)




Fourier order is: 10



Epoch 1

loss: 3.217961  [  512/25000]
loss: 2.748842  [10752/25000]
loss: 2.314478  [20992/25000]

epoch avg train loss: 2.692370   epoch avg train accuracy: 0.211440

-------------------------------

Epoch 2

loss: 2.283867  [  512/25000]
loss: 2.192767  [10752/25000]
loss: 1.948175  [20992/25000]

epoch avg train loss: 2.102938   epoch avg train accuracy: 0.367840

-------------------------------

Epoch 3

loss: 1.876649  [  512/25000]
loss: 1.931707  [10752/25000]
loss: 1.787562  [20992/25000]

epoch avg train loss: 1.832597   epoch avg train accuracy: 0.450880

-------------------------------

Epoch 4

loss: 1.742038  [  512/25000]
loss: 1.660178  [10752/25000]
loss: 1.667130  [20992/25000]

epoch avg train loss: 1.622900   epoch avg train accuracy: 0.511000

-------------------------------

Epoch 5

loss: 1.504452  [  512/25000]
loss: 1.453395  [10752/25000]
loss: 1.340576  [20992/25000]

epoch avg train loss: 1.450292   epoch avg train accuracy: 0.5610

In [ ]:
# seed RNGs
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)

# create datasets
train_data = QuickdrawDataset(train_imgs, train_nums, is_test=False)
# eval_fourier_data = QuickdrawDataset(test_imgs, test_nums, is_test=False)
test_data = QuickdrawDataset(test_imgs, test_nums, is_test=True)

# create dataloaders
g = torch.Generator()
g.manual_seed(RAND_SEED)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
# eval_fourier_loader = DataLoader(eval_fourier_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

# init model and optimizer
model = MLP()
# checkpoint = torch.load(LOAD_PATH, map_location=torch.device(DEVICE))
# model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# optim.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
epoch = 0
print("\n\n\nFourier order is: "+str(FOURIER_ORDER)+"\n\n\n")

# train for EPOCHS number of epochs then evaluate on test data with affine transformations
eval_loop(dataloader=test_loader,model=model)
SAVE_PATH = '/content/drive/My Drive/Fourier/Saved Models/Subset MLP vector single-wide.pt'
for i in range(epoch, EPOCHS):
    print("Epoch " + str(i + 1) + "\n")
    train_loop(dataloader=train_loader,model=model,loss_fn=LOSS_FN,optimizer=optim)
    # eval_loop(dataloader=eval_fourier_loader,model=model)
    torch.save({
                'epoch': i + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optim.state_dict()
                }, SAVE_PATH)
    print("\n-------------------------------\n")
random.seed(RAND_SEED)
accuracies = []
for i in range(30):
  accuracies.append(eval_loop(dataloader=test_loader,model=model))
accuracies = np.asarray(accuracies)
mean = np.mean(accuracies)
std = np.std(accuracies)
print(mean)
print(std)




Fourier order is: 10



Epoch 1

loss: 3.330191  [  512/28000]
loss: 2.760625  [10752/28000]
loss: 2.461224  [20992/28000]

epoch avg train loss: 2.749492   epoch avg train accuracy: 0.209893

-------------------------------

Epoch 2

loss: 2.394397  [  512/28000]
loss: 2.193367  [10752/28000]
loss: 2.275110  [20992/28000]

epoch avg train loss: 2.171338   epoch avg train accuracy: 0.359036

-------------------------------

Epoch 3

loss: 2.088549  [  512/28000]
loss: 1.925457  [10752/28000]
loss: 1.866967  [20992/28000]

epoch avg train loss: 1.900197   epoch avg train accuracy: 0.444000

-------------------------------

Epoch 4

loss: 1.668187  [  512/28000]
loss: 1.670239  [10752/28000]
loss: 1.681529  [20992/28000]

epoch avg train loss: 1.696758   epoch avg train accuracy: 0.499286

-------------------------------

Epoch 5

loss: 1.543532  [  512/28000]
loss: 1.510883  [10752/28000]
loss: 1.487572  [20992/28000]

epoch avg train loss: 1.532818   epoch avg train accuracy: 0.5431